In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm as tqdm

In [6]:
# load all nih files into one big dataframe
#collect the following columns, where file id is the last 3 digits before the .csv 
#    'Sensor Data ID', 'file_id' 
#    'Yaw (rad)', 'Pitch (rad)', 'Roll (rad)', 'Rotation Rate X (rad/s)',
#    'Rotation Rate Y (rad/s)', 'Rotation Rate Z (rad/s)',
#    'User Acceleration X (m/s^2)', 'User Acceleration Y (m/s^2)',
#    'User Acceleration Z (m/s^2)', 'Altitude (m)', 'Speed (m/s)'
for file in tqdm(os.listdir('../data/nih/')):
    if file.endswith('.csv'):
        df = pd.read_csv('../data/nih/' + file)
        df['file_id'] = file.split('.')[0][-3:]
        df = df[['Sensor Data ID', 'file_id', 'Yaw (rad)', 'Pitch (rad)', 'Roll (rad)', 'Rotation Rate X (rad/s)',
                 'Rotation Rate Y (rad/s)', 'Rotation Rate Z (rad/s)', 'User Acceleration X (m/s^2)',
                 'User Acceleration Y (m/s^2)', 'User Acceleration Z (m/s^2)', 'Altitude (m)','Horizontal Accuracy (m)', 'Speed (m/s)']]
        if 'big_df' in locals():
            big_df = pd.concat([big_df, df])
        else:
            big_df = df


  0%|          | 0/40 [00:00<?, ?it/s]C:\Users\gabri\AppData\Local\Temp\ipykernel_16872\3068212805.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/nih/' + file)
  0%|          | 0/40 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [63]:
# load nih_train.npy and nih_val.npy and nih_test.npy to get the file ids
train_nih = np.load('../data/processed_datasets/nih_train.npy', allow_pickle=True)
val_nih = np.load('../data/processed_datasets/nih_val.npy', allow_pickle=True)
test_nih = np.load('../data/processed_datasets/nih_test.npy', allow_pickle=True)

train_ids  = np.unique(train_nih[:, :, -3])
val_ids = np.unique(val_nih[:, :, -3])
test_ids = np.unique(test_nih[:, :, -3])

# create train/val/test dataframes
train_df = big_df[big_df['file_id'].isin(train_ids)]
val_df = big_df[big_df['file_id'].isin(val_ids)]
test_df = big_df[big_df['file_id'].isin(test_ids)]

#drop na
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

# reset index
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# save train/val/test dataframes
train_df.to_parquet('../data/processed_datasets/nih_ssl_train.parquet', index=False)
val_df.to_parquet('../data/processed_datasets/nih_ssl_val.parquet', index=False)
test_df.to_parquet('../data/processed_datasets/nih_ssl_test.parquet', index=False)

In [7]:
# find start indices, which are determined by looking at windows of length 128 with sliding window of legth 64 in unique file_ids (column 1), store 
start_indices = []
#create temporary column to store the index of each row
for file_id in tqdm(train_df['file_id'].unique()):
    file_data = train_df[train_df['file_id'] == file_id]
    start_index = file_data.index[0]

    for i in range(0, len(file_data), 64):
        if i+128 <= len(file_data):
            try:
                start_indices.append(i+start_index)
            except:
                print(i+start_index)


100%|██████████| 5/5 [00:01<00:00,  4.70it/s]


In [9]:
start_indices[-1]

3584886

In [10]:
train_df.iloc[3584886:3584886+128, :]

,Sensor Data ID,file_id,Yaw (rad),Pitch (rad),Roll (rad),Rotation Rate X (rad/s),Rotation Rate Y (rad/s),Rotation Rate Z (rad/s),User Acceleration X (m/s^2),User Acceleration Y (m/s^2),User Acceleration Z (m/s^2),Altitude (m),Horizontal Accuracy (m),Speed (m/s)
3584886,54657834,136,-2.267418,-0.035388,0.021525,-0.000320,-0.000168,-0.000366,0.002115,0.000912,-0.000327,26.0,9.648001,0.0
3584887,54657835,136,-2.267411,-0.035431,0.021570,-0.000076,0.000076,-0.001083,-0.001105,-0.000924,0.002250,26.0,9.648001,0.0
3584888,54657857,136,-2.267332,-0.035401,0.021424,-0.000198,-0.000046,-0.001205,0.001831,0.001724,-0.004686,26.0,9.648001,0.0
3584889,54657858,136,-2.267594,-0.035308,0.021665,-0.000198,-0.000046,-0.001328,-0.000699,-0.000149,-0.003253,26.0,9.648001,0.0
3584890,54657859,136,-2.267495,-0.035349,0.021582,-0.000076,-0.000046,-0.001083,-0.001039,-0.000059,-0.001681,26.0,9.648001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585009,54663642,136,-2.266747,-0.036105,0.021508,-0.000015,0.000366,0.000900,-0.001368,-0.002548,0.000077,-20.0,10.720000,0.0
3585010,54663643,136,-2.267330,-0.035668,0.021756,0.000107,0.000244,0.001022,0.002467,-0.000028,0.001456,-20.0,10.720000,0.0
3585011,54663644,136,-2.267307,-0.035704,0.021769,0.000229,-0.000366,0.000534,-0.001976,-0.000102,0.001054,-20.0,10.720000,0.0
3585012,54663645,136,-2.267529,-0.035518,0.021839,-0.000015,0.000488,0.001022,0.003243,-0.000216,0.000543,-20.0,10.720000,0.0


In [4]:
train_df = pd.read_parquet('../data/processed_datasets/nih_ssl_val.parquet')

In [41]:
file_df = train_df[train_df['file_id'] == '143']
# get non reset index of first row
file_df.index[0]

23874295